# Assignment 4 Group 16

## Import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import t
from datetime import datetime

## Load the data from the files

In [ ]:
# load the indexes as dictionary of {ticker: name}
with open('data/_indexes.csv', 'r') as f:
    # skip the first line
    indexes = {
        line.split(',')[1]: line.split(',')[2].strip()
        for line in f.readlines()[1:]
    }

In [ ]:
# load the actual dataset as a dataframe
EuroStoxx50 = pd.read_csv('data/EUROSTOXX50_Dataset.csv', sep=',', index_col=0, parse_dates=True)
EuroStoxx50.head()

In [ ]:
# summarize the dataset
EuroStoxx50.describe()

In [ ]:
# drop the columns that only contain NaN
EuroStoxx50 = EuroStoxx50.dropna(axis=1, how='all')
# for those who have NaN, fill them with the previous value
EuroStoxx50 = EuroStoxx50.ffill()


In [ ]:
# create the log-returns dataframe
returns = np.log(EuroStoxx50/EuroStoxx50.shift(1))
returns = returns.dropna(axis=0, how='all')

returns.head()

# Point 0: Variance-Covariance method for VaR and ES in a linear portfolio

On the 20th of February 2020 we have an equally weighted portfolio made up of the following equities

- Adidas
- Allianz
- Munich Re
- L'Oréal

We compute the daily VaR and ES with a 5y estimation using a t-student distribution with 4 degrees
of freedom ($\nu$).
The notional of the portfolio is 15 million €. We take a significance level of $\alpha = 0.99\%$.

Wherever we have missing data due to differing trading days for each stock we substitute the previous
day's value.

## Select the data 

In [ ]:
# create a dataframe with the relevant time series
df = returns[['ADSGn.DE', 'ALVG.DE', 'MUVGn.DE', 'OREP.PA']]
# set the date to 20th February 2020
valuation_date = datetime(2020, 2, 20)
# only use data prior to the valuation date
df = df[df.index < valuation_date]
# only use the last 5 years
df = df[df.index > valuation_date - pd.DateOffset(years=5)]


## Set the parameters


In [ ]:
# set nu and alpha
nu = 4
alpha = 0.99
notional = 15 * 10**6

# estimate the mean vector
mean_df = df.mean()
# estimate the covariance matrix
Cov_df = df.cov()
# create the weights vector
weights = np.array([0.25, 0.25, 0.25, 0.25])

## Daily VaR
We compute the daily VaR using the variance-covariance method. The daily VaR is given by:

$$
VaR_{\alpha} = \underbrace{\bar\mu \cdot \bar\omega}_{\mu} + \underbrace{ \sqrt{\bar\omega^T \Sigma \bar\omega}}_{\sigma} \cdot t^{-1}_{\nu} (\alpha)
$$

Where:

- $\bar\omega$ is the vector of weights of the portfolio
- $\bar\mu$ is the vector of expected returns of the portfolio
- $\Sigma$ is the variance-covariance matrix of the returns of the portfolio
- $t^{-1}_{\nu}(\alpha)$ is the $\alpha$-quantile of the t-student distribution with $\nu$ degrees of freedom

To compute the quantity $t^{-1}_{\nu}(\alpha)$ we use the `t.ppf` function from the `scipy.stats` module.

See [this stackoverflow answer](https://stackoverflow.com/questions/65468026/norm-ppf-vs-norm-cdf-in-pythons-scipy-stats)
and [this documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html#:~:text=ppf(q%2C%20df,cdf%20%E2%80%94%20percentiles).) for more information.


In [ ]:
# find the t_alpha quantile
t_alpha = t.ppf(alpha, nu)

# compute the VaR
VaR = mean_df @ weights + np.sqrt(weights @ Cov_df @ weights) * t_alpha

print(f'The daily VaR at 99% confidence level is {VaR:.2%}')
print(f'The daily VaR at 99% confidence level is {VaR * notional:.2f} EUR')

### Daily ES

We compute the daily ES using the variance-covariance method. The daily ES is given by:

$$
ES_{\alpha} = \bar\mu \cdot \bar\omega +
    \sqrt{\bar\omega^T \Sigma \bar\omega} \cdot
    \underbrace{
        \frac{\nu + ( t^{-1}_{\nu}(\alpha) )^2}{\nu - 1} \cdot \frac{ \phi_{\nu} (t^{-1}_{\nu}(\alpha)) }{1 - \alpha}
    }_{ES_{\alpha}^{std}}
$$

Where:

- $\bar\omega$ is the vector of weights of the portfolio
- $\bar\mu$ is the vector of expected returns of the portfolio
- $\Sigma$ is the variance-covariance matrix of the returns of the portfolio
- $t^{-1}_{\nu}(\alpha)$ is the $\alpha$-quantile of the t-student distribution with $\nu$ degrees of freedom
- $\phi_{\nu} (\cdot)$ is the density function of the t-student distribution with $\nu$ degrees of freedom


In [ ]:
# compute the ES for the standard t-distribution
ES_std = (nu + t_alpha**2) / (nu - 1) * (t.pdf(t_alpha, nu) / (1 - alpha))

# compute the ES for the portfolio
ES = mean_df @ weights + np.sqrt(weights @ Cov_df @ weights) * ES_std

print(f'The daily ES at 99% confidence level is {ES:.2%}')
print(f'The daily ES at 99% confidence level is {ES * notional:.2f} EUR')

# Point 1: Historical simulation, bootstrap and PCA for VaR and ES in a linear portfolio

On the 20th of March 2019 we must compute the following quantitities with $\alpha = 0.95\%$:

- Portfolio 1: AXA (20K shares), Sanofi (20K shares), Volkswagen (10K shares).
    We compute the daily VaR and ES with a Historical Simulation and Bootstrap method (with 200 simulations) and a 5 years estimation.
- Portfolio 2: Adidas, Airbus, BBVA, BMW and Deutsche Telekom (all equally weighted).
    We compute the daily VaR and ES with a 5 year estimation using a Weighted Historical Simulation with $\lambda = 0.95$.
- Portfolio 3: An equally weighted portfolio with shares of the first 18 companies.
    We compute the 10 days VaR and ES with a 5 year estimation using a Gaussian parametric PCA approach using the first n principanl components (with n = 1, 2, 3, 4, 5).

For each portfolio we also check the Plausibility Check.


## Data setup
We set the parameters for the various models and select the data to use

In [ ]:
# set the parameters
alpha = 0.95
lmd = 0.95 # lambda is a reserved keyword
# set the valuation date to 20th March 2019
valuation_date = datetime(2019, 3, 20)

# select only the relevant returns
df = returns[returns.index <= valuation_date]
# only use the last 5 years
df = df[df.index >= valuation_date - pd.DateOffset(years=5)]

## Point 1.1 Portfolio 1
First of all we set up the weights we will use to compute the various quantities.

In [ ]:
tot_shares = 50
w_1 = np.array([20/tot_shares, 20/tot_shares, 10/tot_shares])
indexes.get('AXA')

### Historical Simulation
We freeze the portfolio to the valutaion date and write the loss function as a function of the weights and the returns.

In [ ]:
# compute the value of the portfolio

## Point 1.B

In [ ]:
# Build an equally weighted portfolio
w_2 = 0.20
# create the array with the weights
ptf_weights = np.array([w_2, w_2, w_2, w_2, w_2])
# normalization factor
C  = (1 - lmd) / (1 - lmd**len(df))
# compute the decreasing sequence of weights: w_s = C*lambda^(t-s)
weights = np.array([C * lmd**(len(df) - t) for t in range(len(df))])


# extract the log returns of the following companies Adidas, Airbus, BBVA, BMW, Deutsche 
tickers_of_interest = ['ADSGn.DE', 'AIR.PA', 'BBVA.MC', 'BMWG.DE', 'DTEGn.DE']

# Selecting columns for tickers of interest
selected_returns = df[tickers_of_interest]

selected_returns.head()

print(weights)

In [ ]:

# the loss distribution
loss  = - 1 * (selected_returns.T @ weights)

# sort the loss in ascending order
loss_sorted = sorted(loss, reverse=True)

print(loss_sorted)

#var = loss_sorted[int(len(loss_sorted) * alpha)]
var = max([
    loss_sorted[i]
    for i in range(len(loss_sorted))
    if np.sum(weights[:i]) <= 1 - alpha
])

print(var)

print(f'The VaR at 95% confidence level is {var:.2%}')

es = np.sum(loss_sorted) / np.sum(weights)

print(f'The ES at 95% confidence level is {es:.2%}')
